# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [6]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import sqlalchemy
from sqlalchemy import Integer, String, MetaData, Table, Column, create_engine, text, inspect, select

import warnings
warnings.filterwarnings("ignore") #supress warnings

/var/folders/7x/mfyhhlkn0fz774bmn50_szsm0000gn/T/ipykernel_24013/1746294441.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import Data

In [7]:
# API 
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# request for api 
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()
    
    # data update check 
    last_modified_header = response.headers.get('Last-Modified')
    
    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to df 
    hospital_df = pd.DataFrame(data)
    
    print(hospital_df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [8]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [9]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
1380,1381,1017,912155626,Umass Memorial Medical Center,Umass Memorial Medical Center,55 Lake Ave North,Worcester,MA,01655,25027,325,N,Y,N,Y,220163,Worcester County,>299 beds,"November 20, 2023"
2878,2879,2265,626002428,Hardin Medical Center,Hardin Medical Center,935 Wayne Road,Savannah,TN,38372,47071,58,N,N,N,N,440109,Hardin County,<100 beds,"November 20, 2023"
2723,2724,2026,208295721,Upmc East Hospital,Upmc East Hospital,2775 Mosside Blvd,Monroeville,PA,15146,42003,155,N,Y,N,N,390328,Allegheny County,100-299 beds,"November 20, 2023"
3300,3301,2520,550562976,Grant Memorial Hospital,Grant Memorial Hospital,117 Hospital Drive,Petersburg,WV,26847,54023,45,N,N,N,Y,511316,Grant County,<100 beds,"November 20, 2023"
138,139,97,261214372,Nea Baptist Memorial Hospital,Nea Baptist Memorial Hospital,4800 East Johnson Avenue,Jonesboro,AR,72401,05001,228,N,N,N,Y,040118,Arkansas County,100-299 beds,"November 20, 2023"
1749,1750,1355,430738490,Ssm Cardinal Glennon Childrens Hospit,,1465 S Grand Blvd,Saint Louis,MO,63104,29510,190,N,Y,Y,Y,263300,St. Louis city,100-299 beds,"November 20, 2023"
1191,1192,906,720411322,St Patrick Hospital,St Patrick Hospital,524 Dr Michael Debakey Drive,Lake Charles,LA,70601,22019,230,N,Y,N,N,190027,Calcasieu Parish,100-299 beds,"November 20, 2023"
544,545,371,590675143,Flagler Hospital,Flagler Hospital,Street: 400 Health Park Blvd,St Augustine,FL,32086,12109,335,N,Y,N,N,100090,St. Johns County,>299 beds,"November 20, 2023"
1470,1471,1139,381218516,Mclaren Macomb,Mclaren Macomb,1000 Harrington,Mt Clemens,MI,48043,26099,288,N,Y,N,Y,230227,Macomb County,100-299 beds,"November 20, 2023"
3188,3189,2446,546075662,Twin County Regional Hospital,Twin County Regional Hospital,200 Hospital Drive,Galax,VA,24333,51640,141,N,N,N,Y,490115,Galax city,100-299 beds,"November 20, 2023"


## Data Inspection

In [10]:
hospital_df.shape 

(3491, 19)

In [24]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [25]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [26]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [27]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [28]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [11]:
# Hospital Information Table

# create unique ID for address
hospital_df['address_id'] = hospital_df['street_address'] + '_' + hospital_df['city'] + '_' + hospital_df['state'] + '_' + hospital_df['zip_code'] + '_' + hospital_df['fips_state_and_county_code']
hospital_df['address_id'] = hospital_df['address_id'].rank(method='dense', ascending=False).astype(int)

#create unique ID for hospital capacity
hospital_df['capacity_id'] = hospital_df['hospital_id'].astype(str) + '_' + hospital_df['hospital_bed_count'].astype(str) + '_' + hospital_df['hospital_bed_size']
hospital_df['capacity_id'] = hospital_df['capacity_id'].rank(method='dense', ascending=False).astype(int)


# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,010007,609,2381,"November 20, 2023"
1,2,2,St Vincents East,630578923,010011,1075,1270,"November 20, 2023"
2,3,3,Shelby Baptist Medical Center,630312913,010016,3323,667,"November 20, 2023"
3,4,4,Callahan Eye Foundation Hosp,630459034,010018,2440,556,"November 20, 2023"
4,5,5,Cherokee Medical Center,581973570,010022,1337,445,"November 20, 2023"
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,949,730,"November 20, 2023"
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,718,729,"November 20, 2023"
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,2901,728,"November 20, 2023"
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,2967,726,"November 20, 2023"


In [12]:
hospital_df

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,...,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt,address_id,capacity_id
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,...,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023",609,2381
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,...,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023",1075,1270
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,...,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023",3323,667
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,...,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023",2440,556
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,...,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023",1337,445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,...,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023",949,730
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,...,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023",718,729
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,...,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023",2901,728
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,...,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023",2967,726


In [13]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,N,"November 20, 2023"
1,2,N,"November 20, 2023"
2,3,N,"November 20, 2023"
3,4,N,"November 20, 2023"
4,5,N,"November 20, 2023"
...,...,...,...
3486,3487,N,"November 20, 2023"
3487,3488,N,"November 20, 2023"
3488,3489,N,"November 20, 2023"
3489,3490,N,"November 20, 2023"


In [14]:
# Hospital Capacity Table - extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,2381,<100 beds,99,"November 20, 2023"
1,1270,>299 beds,362,"November 20, 2023"
2,667,100-299 beds,252,"November 20, 2023"
3,556,100-299 beds,106,"November 20, 2023"
4,445,<100 beds,60,"November 20, 2023"
...,...,...,...,...
3486,730,<100 beds,16,"November 20, 2023"
3487,729,<100 beds,7,"November 20, 2023"
3488,728,<100 beds,63,"November 20, 2023"
3489,726,<100 beds,46,"November 20, 2023"


In [15]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,N,"November 20, 2023"
1,2,N,"November 20, 2023"
2,3,N,"November 20, 2023"
3,4,N,"November 20, 2023"
4,5,N,"November 20, 2023"
...,...,...,...
3486,3487,N,"November 20, 2023"
3487,3488,N,"November 20, 2023"
3488,3489,N,"November 20, 2023"
3489,3490,N,"November 20, 2023"


In [16]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,N,"November 20, 2023"
1,2,Y,"November 20, 2023"
2,3,N,"November 20, 2023"
3,4,Y,"November 20, 2023"
4,5,N,"November 20, 2023"
...,...,...,...
3486,3487,N,"November 20, 2023"
3487,3488,N,"November 20, 2023"
3488,3489,N,"November 20, 2023"
3489,3490,N,"November 20, 2023"


In [11]:
import pymysql.cursors

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password,
                       ssl={"fake_flag_to_enable_tls": True})

In [35]:
# specify the database file path
database_path = "sqlite:///hospital_insights.db"

# create SQLAlchemy Engine 
engine = create_engine(database_path)
conn = engine

# define metadata
metadata = MetaData()
metadata.reflect(bind=engine)


# export the DataFrame to a SQL table
hospital_df.to_sql('HospitalInsights', conn, if_exists='replace', index=False)

3491

In [36]:
# create Hospital Capacity Table with capacity_id as primary key
hospital_capacity_table = Table(
    'HospitalCapacity',
    metadata,
    Column('capacity_id', Integer, primary_key=True),
    Column('hospital_bed_size', String),
    Column('hospital_bed_count', Integer),
    Column('updated_dt', String)
)

# create Children Hospital Table with hospital_id w/out a primary key
children_hospital_table = Table(
    'ChildrenHospital',
    metadata,
    Column('hospital_id', Integer),
    Column('children_hospital_f', Integer),
    Column('updated_dt', String)
)

# create Teaching Table with hospital_id w/out a primary key
teaching_table = Table(
    'Teaching',
    metadata,
    Column('hospital_id', Integer),
    Column('memb_counc_teach_hosps_f', Integer),
    Column('updated_dt', String)
)

metadata.create_all(engine)

# export df to sql table 
hospital_capacity.to_sql('HospitalCapacity', conn, if_exists='replace', index=False) #hospital capacity
children_hospital.to_sql('ChildrenHospital', engine, if_exists='replace', index=False) # children hospital
teaching.to_sql('Teaching', engine, if_exists='replace', index=False) # teaching


3491

In [37]:
# inspect database 
inspector = inspect(engine) # create inspector

# obtain table names 
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Tables in the database:
ChildrenHospital
HospitalCapacity
HospitalInsights
Teaching


In [38]:
# print a sample from each table 

# define function 
def print_sample_data(table, table_name):
    # create SELECT query to obtain sample from each table 
    select_query = select(table).limit(5)
    
    # execute w/connection from engine 
    with engine.connect() as connection:
        result = connection.execute(select_query)
        
        # print sample
        sample_data = result.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)
        print()

# define tables
hospital_capacity_table = Table('HospitalCapacity', metadata, autoload=True, autoload_with=engine)
children_hospital_table = Table('ChildrenHospital', metadata, autoload=True, autoload_with=engine)
teaching_table = Table('Teaching', metadata, autoload=True, autoload_with=engine)

# print sample from each table 
print_sample_data(hospital_capacity_table, 'HospitalCapacity')
print_sample_data(children_hospital_table, 'ChildrenHospital')
print_sample_data(teaching_table, 'Teaching')

Sample data from table 'HospitalCapacity':
(2381, '<100 beds', 99, 'November 20, 2023')
(1270, '>299 beds', 362, 'November 20, 2023')
(667, '100-299 beds', 252, 'November 20, 2023')
(556, '100-299 beds', 106, 'November 20, 2023')
(445, '<100 beds', 60, 'November 20, 2023')

Sample data from table 'ChildrenHospital':
('1', 0, 'November 20, 2023')
('2', 0, 'November 20, 2023')
('3', 0, 'November 20, 2023')
('4', 0, 'November 20, 2023')
('5', 0, 'November 20, 2023')

Sample data from table 'Teaching':
('1', 0, 'November 20, 2023')
('2', 1, 'November 20, 2023')
('3', 0, 'November 20, 2023')
('4', 1, 'November 20, 2023')
('5', 0, 'November 20, 2023')



In [17]:
import sqlite3

# create database file path
database_path = "hospital_insights_2.db"

# connect to database 
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [18]:
create_hospital_capacity_table_query = '''
CREATE TABLE IF NOT EXISTS HospitalCapacity (
    capacity_id INTEGER PRIMARY KEY,
    hospital_bed_size TEXT,
    hospital_bed_count INTEGER,
    updated_dt TEXT
)
'''
cursor.execute(create_hospital_capacity_table_query)

for index, row in hospital_capacity.iterrows():
    insert_hospital_capacity_query = '''
    INSERT INTO HospitalCapacity (hospital_bed_size, hospital_bed_count, updated_dt)
    VALUES (?, ?, ?)
    '''
    cursor.execute(insert_hospital_capacity_query, (row['hospital_bed_size'], row['hospital_bed_count'], row['updated_dt']))

In [21]:
create_children_hospital_table_query = '''
CREATE TABLE IF NOT EXISTS ChildrenHospital (
    hospital_id INTEGER,
    children_hospital_f INTEGER,
    updated_dt TEXT
)
'''

cursor.execute(create_children_hospital_table_query)

for index, row in children_hospital.iterrows():
    insert_children_hospital_query = '''
    INSERT INTO ChildrenHospital (hospital_id, children_hospital_f, updated_dt)
    VALUES (?, ?, ?)
    '''
    cursor.execute(insert_children_hospital_query, (row['hospital_id'], row['children_hospital_f'], row['updated_dt']))

In [22]:
create_teaching_table_query = '''
CREATE TABLE IF NOT EXISTS Teaching (
    hospital_id INTEGER,
    memb_counc_teach_hosps_f INTEGER,
    updated_dt TEXT
)
'''
cursor.execute(create_teaching_table_query)

# Insert data from teaching DataFrame into the Teaching table
for index, row in teaching.iterrows():
    insert_teaching_query = '''
    INSERT INTO Teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
    VALUES (?, ?, ?)
    '''
    cursor.execute(insert_teaching_query, (row['hospital_id'], row['memb_counc_teach_hosps_f'], row['updated_dt']))


In [23]:
conn.commit()